In [17]:
#!/usr/bin/env python3
# pip install geopandas shapely vt2geojson mercantile tqdm pyproj pandas numpy requests

import time, json, requests, mercantile
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.ops import unary_union
from vt2geojson.tools import vt_bytes_to_geojson
from pyproj import CRS, Geod
from tqdm import tqdm

# ─────────────────────── CONFIG ────────────────────────
API_KEY    = "BDxJkUVBgwiy5yhJyLyf"     # tu key MapTiler
CENTER     = (14.6398861, -90.5496683)  # lat, lon
RADIUS_M   = 1000                       # metros
ZOOM       = 14                         # nivel de tesela
LAYER      = "transportation"           # ejes viales

TMP        = Path("ejes.geojson")
OUT_UTM    = Path("calzada_buffer.geojson")
OUT_WGS84  = Path("calzada_buffer_wgs84.geojson")

# CRS UTM (ajusta a tu zona)
crs_utm = CRS.from_epsg(32615)  # Guatemala 15N

# Anchos por defecto si no hay width/lanes
DEFAULT_WIDTH = {
    "motorway":32, "trunk":26, "primary":22, "secondary":18,
    "tertiary":14, "residential":10, "service":8, "footway":4
}
LANE_WIDTH = 3.5  # m por carril

# ───────────────────── DESCARGA TESSELAS ────────────────────
t0 = time.perf_counter()

# 1) bbox lon/lat
geod = Geod(ellps="WGS84")
lons,lats = [],[]
for az in (0,90,180,270):
    lon, lat, _ = geod.fwd(CENTER[1], CENTER[0], az, RADIUS_M)
    lons.append(lon); lats.append(lat)
west, south, east, north = min(lons), min(lats), max(lons), max(lats)

# 2) descargar vector tiles
tiles = list(mercantile.tiles(west,south,east,north,[ZOOM]))
feats = []
for t in tqdm(tiles, desc="Descargando"):
    url = f"https://api.maptiler.com/tiles/v3/{t.z}/{t.x}/{t.y}.pbf?key={API_KEY}"
    pbf = requests.get(url, timeout=15).content
    feats += vt_bytes_to_geojson(pbf, t.x, t.y, t.z, layer=LAYER)["features"]

TMP.write_text(json.dumps({"type":"FeatureCollection","features":feats}, ensure_ascii=False))
print(f"↳ Descarga: {time.perf_counter()-t0:0.2f}s")

# ───────────────────── GEN BUFFER ──────────────────────────
t1 = time.perf_counter()

# 3.1) Leer + reproyectar
gdf = gpd.read_file(TMP).to_crs(crs_utm)
idx = gdf.index

# 3.2) Series de width y lanes garantizadas
width_raw = gdf["width"]   if "width" in gdf.columns else pd.Series(index=idx, dtype=float)
lanes_raw = gdf["lanes"]   if "lanes" in gdf.columns else pd.Series(index=idx, dtype=float)
width_s   = pd.to_numeric(width_raw, errors="coerce").reindex(idx)
lanes_s   = pd.to_numeric(lanes_raw, errors="coerce").reindex(idx)
class_s   = gdf.get("class", pd.Series("unknown", index=idx))

# 3.3) Estimar ancho base (m)
gdf["computed_width"] = (
    width_s
    .fillna(lanes_s * LANE_WIDTH)
    .fillna(class_s.map(DEFAULT_WIDTH))
    .fillna(8)
)

# 3.4) Percentiles para datos reales
p10, p50, p90 = gdf["computed_width"].quantile([0.10, 0.50, 0.90])
print(f"Percentiles width → 10%:{p10:.1f}, 50%:{p50:.1f}, 90%:{p90:.1f}")

# 3.5) Shrink más generoso: [0.95,0.85,0.75], clip mínimo 0.75
edges   = [p10, p50, p90]
factors = [0.95, 0.85, 0.75]
gdf["shrink_factor"] = np.interp(gdf["computed_width"], edges, factors).clip(0.75, 0.95)

# 3.6) Radio de buffer = ancho * shrink / 2
radii = gdf["computed_width"] * gdf["shrink_factor"] / 2

# 3.7) Crear buffer redondeado
gdf["buffer_geom"] = gdf.geometry.buffer(
    radii, cap_style=1, join_style=1, resolution=8
)

# 3.8) Unión en paralelo (chunks)
chunks = [gdf["buffer_geom"][i:i+400] for i in range(0, len(gdf), 400)]
with ThreadPoolExecutor() as pool:
    parts = list(tqdm(pool.map(unary_union, chunks), total=len(chunks), desc="Uniendo"))
calzada = unary_union(parts)

# 3.9) Sellado y simplificación leve
calzada = (
    calzada
    .buffer(0.3, resolution=4)
    .buffer(-0.3, resolution=4)
    .simplify(0.02)
)

print(f"↳ Buffer+union: {time.perf_counter()-t1:0.2f}s")

# ───────────────────── EXPORTACIÓN ──────────────────────────
gpd.GeoSeries([calzada], crs=crs_utm).to_file(OUT_UTM, driver="GeoJSON")
gpd.GeoSeries([calzada], crs=crs_utm).to_crs(epsg=4326).to_file(OUT_WGS84, driver="GeoJSON")

print("✅ UTM  →", OUT_UTM.resolve())
print("✅ WGS84→", OUT_WGS84.resolve())
print(f"⏱ Total: {time.perf_counter()-t0:0.1f}s")


Descargando: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


↳ Descarga: 1.66s
Percentiles width → 10%:8.0, 50%:8.0, 90%:26.0


Uniendo: 100%|███████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.99it/s]


↳ Buffer+union: 2.22s
✅ UTM  → C:\Proyectos\Mexico\Cartografia\mexico-claro-cartography\Claro\Cartografia\calzada_buffer.geojson
✅ WGS84→ C:\Proyectos\Mexico\Cartografia\mexico-claro-cartography\Claro\Cartografia\calzada_buffer_wgs84.geojson
⏱ Total: 4.9s


# Codigo dinamico

In [3]:
#!/usr/bin/env python3
# pip install geopandas shapely vt2geojson mercantile tqdm pyproj pandas numpy requests

import time
import math
import json
import requests
import mercantile
import numpy as np
import pandas as pd
import geopandas as gpd

from pathlib import Path
from tqdm import tqdm
from shapely.geometry import LineString, MultiLineString
from shapely.ops import unary_union
from concurrent.futures import ThreadPoolExecutor
from vt2geojson.tools import vt_bytes_to_geojson
from pyproj import CRS, Geod

# ─────────────────────── CONFIG ────────────────────────
API_KEY    = "BDxJkUVBgwiy5yhJyLyf"      # tu key MapTiler
CENTER     = (14.6398861, -90.5496683)   # lat, lon
RADIUS_M   = 1000                        # metros
ZOOM       = 14                          # nivel de tesela
LAYER      = "transportation"            # ejes viales

TMP        = Path("ejes.geojson")
OUT_UTM    = Path("calzada_buffer.geojson")
OUT_WGS84  = Path("calzada_buffer_wgs84.geojson")

# CRS UTM (ajusta a tu zona)
crs_utm = CRS.from_epsg(32615)  # Guatemala 15N

# Valores por defecto de ancho (m) si no hay width/lanes
DEFAULT_WIDTH = {
    "motorway":32, "trunk":26, "primary":22, "secondary":18,
    "tertiary":14, "residential":10, "service":8, "footway":4
}
LANE_WIDTH = 3.5  # m por carril

# ─────────────────── Función de densificación ───────────────────
def densify_curvas(line, max_seg_len=5.0, angle_thresh=8.0):
    """
    Subdivide tramos de LineString donde la curvatura (>angle_thresh)
    o la longitud (>max_seg_len) lo requieran.
    """
    coords = list(line.coords)
    new_coords = [coords[0]]
    for i in range(1, len(coords)-1):
        p0, p1, p2 = coords[i-1], coords[i], coords[i+1]
        v1 = (p1[0]-p0[0], p1[1]-p0[1])
        v2 = (p2[0]-p1[0], p2[1]-p1[1])
        dot = v1[0]*v2[0] + v1[1]*v2[1]
        mag = math.hypot(*v1) * math.hypot(*v2)
        ang = math.degrees(math.acos(max(min(dot/mag,1),-1))) if mag>0 else 0
        seg_len = LineString([p0,p1]).length
        if ang > angle_thresh or seg_len > max_seg_len:
            n = int(math.ceil(seg_len / max_seg_len))
            for k in range(1, n+1):
                t = k/(n+1)
                xi = p0[0] + (p1[0]-p0[0])*t
                yi = p0[1] + (p1[1]-p0[1])*t
                new_coords.append((xi, yi))
        new_coords.append(p1)
    new_coords.append(coords[-1])
    return LineString(new_coords)

def densify_geometry(geom):
    """
    Aplica densify_curvas a LineString o a cada parte de MultiLineString.
    Devuelve una única geometría densificada.
    """
    if isinstance(geom, LineString):
        return densify_curvas(geom)
    elif isinstance(geom, MultiLineString):
        parts = [densify_curvas(ls) for ls in geom.geoms]
        return unary_union(parts)
    else:
        # Si no es línea, devolver tal cual
        return geom

# ───────────────────── DESCARGA TESSELAS ────────────────────
t0 = time.perf_counter()

# 1) Calcular bounding box
geod = Geod(ellps="WGS84")
lons, lats = [], []
for az in (0,90,180,270):
    lon, lat, _ = geod.fwd(CENTER[1], CENTER[0], az, RADIUS_M)
    lons.append(lon); lats.append(lat)
west, south, east, north = min(lons), min(lats), max(lons), max(lats)

# 2) Descargar vector tiles
tiles = list(mercantile.tiles(west, south, east, north, [ZOOM]))
features = []
for t in tqdm(tiles, desc="Descargando"):
    url = f"https://api.maptiler.com/tiles/v3/{t.z}/{t.x}/{t.y}.pbf?key={API_KEY}"
    pbf = requests.get(url, timeout=15, verify=False).content
    features += vt_bytes_to_geojson(pbf, t.x, t.y, t.z, layer=LAYER)["features"]

TMP.write_text(json.dumps({"type":"FeatureCollection","features":features},
                           ensure_ascii=False))
print(f"↳ Descarga: {time.perf_counter()-t0:0.2f}s")

# ───────────────────── GENERAR BUFFER ──────────────────────────
t1 = time.perf_counter()

# 3.1) Leer y reproyectar
gdf = gpd.read_file(TMP).to_crs(crs_utm)
idx = gdf.index

# 3.2) Preparar width/lanes
width_raw = gdf["width"] if "width" in gdf.columns else pd.Series(index=idx, dtype=float)
lanes_raw = gdf["lanes"] if "lanes" in gdf.columns else pd.Series(index=idx, dtype=float)
width_s   = pd.to_numeric(width_raw, errors="coerce").reindex(idx)
lanes_s   = pd.to_numeric(lanes_raw, errors="coerce").reindex(idx)
class_s   = gdf.get("class", pd.Series("unknown", index=idx))

# 3.3) Estimar ancho base
gdf["computed_width"] = (
    width_s
    .fillna(lanes_s * LANE_WIDTH)
    .fillna(class_s.map(DEFAULT_WIDTH))
    .fillna(8)
)

# 3.4) Percentiles para shrink dinámico
p10, p50, p90 = gdf["computed_width"].quantile([0.10, 0.50, 0.90])
print(f"Percentiles width → 10%:{p10:.1f}, 50%:{p50:.1f}, 90%:{p90:.1f}")

# 3.5) Interpolar shrink_factor
edges   = [p10, p50, p90]
factors = [0.95, 0.85, 0.75]
gdf["shrink_factor"] = np.interp(gdf["computed_width"], edges, factors).clip(0.75, 0.95)

# 3.6) Densificar + buffer
buffers = []
for _, row in tqdm(gdf.iterrows(), total=len(gdf), desc="Buffering"):
    geom = densify_geometry(row.geometry)
    radio = row.computed_width * row.shrink_factor / 2.0
    # resolución proporcional al ancho
    res = int(np.interp(radio*2, [2,30], [8,20]))
    buf = geom.buffer(radio, cap_style=1, join_style=1, resolution=res)
    buffers.append(buf)

# 3.7) Unión paralela
chunks = [buffers[i:i+400] for i in range(0, len(buffers), 400)]
with ThreadPoolExecutor() as pool:
    partes = list(tqdm(pool.map(unary_union, chunks),
                       total=len(chunks), desc="Uniendo"))
calzada = unary_union(partes)

# 3.8) Sellado y simplificación
calzada = (
    calzada
    .buffer(0.3, resolution=4)
    .buffer(-0.3, resolution=4)
    .simplify(0.02)
)

print(f"↳ Buffer+union: {time.perf_counter()-t1:0.2f}s")

# ───────────────────── EXPORTACIÓN ──────────────────────────
gpd.GeoSeries([calzada], crs=crs_utm).to_file(OUT_UTM, driver="GeoJSON")
gpd.GeoSeries([calzada], crs=crs_utm).to_crs(epsg=4326).to_file(OUT_WGS84, driver="GeoJSON")

print("✅ UTM  →", OUT_UTM.resolve())
print("✅ WGS84→", OUT_WGS84.resolve())
print(f"⏱ Total: {time.perf_counter()-t0:0.1f}s")


Descargando:   0%|                                                                              | 0/4 [00:00<?, ?it/s]C:\Users\CA987YS\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.maptiler.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Descargando:  25%|█████████████████▌                                                    | 1/4 [00:01<00:05,  1.86s/it]C:\Users\CA987YS\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.maptiler.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Descargando:  50%|███████████████████████████████████                                   | 2/4 [00:03<0

↳ Descarga: 5.70s
Percentiles width → 10%:8.0, 50%:8.0, 90%:22.0


Uniendo: 100%|██████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.59it/s]


↳ Buffer+union: 40.85s
✅ UTM  → C:\Users\CA987YS\01. Proyectos\Cartografia\calzada_buffer.geojson
✅ WGS84→ C:\Users\CA987YS\01. Proyectos\Cartografia\calzada_buffer_wgs84.geojson
⏱ Total: 50.5s


# Mejora

In [5]:
#!/usr/bin/env python3
# pip install geopandas shapely vt2geojson mercantile tqdm pyproj pandas numpy requests

import time, math, json, requests, mercantile
import numpy as np, pandas as pd, geopandas as gpd
from pathlib import Path
from tqdm import tqdm
from shapely.geometry import LineString, MultiLineString
from shapely.ops import unary_union
from concurrent.futures import ThreadPoolExecutor
from vt2geojson.tools import vt_bytes_to_geojson
from pyproj import CRS, Geod

# ───────────────────────── CONFIG ─────────────────────────
API_KEY  = "BDxJkUVBgwiy5yhJyLyf"          # MapTiler
CENTER   = (14.6398861, -90.5496683)       # lat, lon
RADIUS_M = 1000                            # m
ZOOM     = 15                              # ↑ mayor detalle
LAYER    = "transportation"                # capa vial
TMP       = Path("ejes.geojson")
OUT_UTM   = Path("calzada_buffer.geojson")
OUT_WGS84 = Path("calzada_buffer_wgs84.geojson")
crs_utm   = CRS.from_epsg(32615)           # Guatemala 15 N

DEFAULT_WIDTH = {                          # ancho por clase OSM
    "motorway":32,"trunk":26,"primary":22,"secondary":18,
    "tertiary":14,"residential":10,"service":8,"footway":4
}
LANE_WIDTH = 3.5   # m/carril

# ───────────── DENSIFICACIÓN ADAPTATIVA ─────────────
def densify_curvas(line, max_seg_len=3.0, angle_thresh=5.0):
    coords = list(line.coords); new = [coords[0]]
    for i in range(1, len(coords)-1):
        p0, p1, p2 = coords[i-1], coords[i], coords[i+1]
        v1 = (p1[0]-p0[0], p1[1]-p0[1]); v2 = (p2[0]-p1[0], p2[1]-p1[1])
        dot = v1[0]*v2[0] + v1[1]*v2[1]
        mag = math.hypot(*v1) * math.hypot(*v2)
        ang = math.degrees(math.acos(max(min(dot/mag, 1), -1))) if mag else 0
        seg_len = LineString([p0, p1]).length
        if ang > angle_thresh or seg_len > max_seg_len:
            n = int(math.ceil(seg_len / max_seg_len))
            for k in range(1, n+1):
                t = k / (n+1)
                new.append((p0[0] + (p1[0]-p0[0])*t,
                            p0[1] + (p1[1]-p0[1])*t))
        new.append(p1)
    new.append(coords[-1])
    return LineString(new)

def densify_geometry(geom):
    if isinstance(geom, LineString):
        return densify_curvas(geom)
    elif isinstance(geom, MultiLineString):
        return unary_union([densify_curvas(ls) for ls in geom.geoms])
    return geom

# ────────────────── DESCARGA TESSELAS ──────────────────
t0 = time.perf_counter()
geod = Geod(ellps="WGS84")
lons, lats = [], []
for az in (0, 90, 180, 270):
    lon, lat, _ = geod.fwd(CENTER[1], CENTER[0], az, RADIUS_M)
    lons.append(lon); lats.append(lat)
west, south, east, north = min(lons), min(lats), max(lons), max(lats)

tiles = list(mercantile.tiles(west, south, east, north, [ZOOM]))
features = []
for t in tqdm(tiles, desc="Descargando"):
    url = f"https://api.maptiler.com/tiles/v3/{t.z}/{t.x}/{t.y}.pbf?key={API_KEY}"
    pbf = requests.get(url, timeout=15, verify=False).content
    features += vt_bytes_to_geojson(pbf, t.x, t.y, t.z, layer=LAYER)["features"]

TMP.write_text(json.dumps({"type": "FeatureCollection", "features": features},
                          ensure_ascii=False))
print(f"↳ Descarga: {time.perf_counter()-t0:0.1f}s")

# ────────────────── GENERAR BUFFER ──────────────────
t1 = time.perf_counter()
gdf = gpd.read_file(TMP).to_crs(crs_utm)
idx = gdf.index                                    # índice base

# --- columnas width & lanes convertidas a Series ---
width_raw = gdf["width"] if "width" in gdf.columns else pd.Series(index=idx, dtype=float)
lanes_raw = gdf["lanes"] if "lanes" in gdf.columns else pd.Series(index=idx, dtype=float)

width_s = pd.to_numeric(width_raw, errors="coerce")
lanes_s = pd.to_numeric(lanes_raw, errors="coerce")
# por si llegan escalares
if not isinstance(width_s, pd.Series):
    width_s = pd.Series(width_s, index=idx)
if not isinstance(lanes_s, pd.Series):
    lanes_s = pd.Series(lanes_s, index=idx)

class_s = gdf.get("class").reindex(idx)

gdf["computed_width"] = (
    width_s.fillna(lanes_s * LANE_WIDTH)
           .fillna(class_s.map(DEFAULT_WIDTH))
           .fillna(8)
)

# --- factor de encogimiento dinámico ---
p10, p50, p90 = gdf["computed_width"].quantile([.10, .50, .90])
edges, factors = [p10, p50, p90], [0.95, 0.85, 0.78]
gdf["shrink_factor"] = np.interp(gdf["computed_width"], edges, factors).clip(0.78, 0.95)

# --- buffer pieza a pieza ---
buffers = []
for _, row in tqdm(gdf.iterrows(), total=len(gdf), desc="Buffering"):
    geom = densify_geometry(row.geometry)
    radio = row.computed_width * row.shrink_factor / 2
    res = int(np.clip(np.interp(radio*2, [2, 40], [8, 32]), 8, 32))
    buf = geom.buffer(
        radio,
        cap_style="flat",
        join_style="mitre",
        mitre_limit=3.0,
        quad_segs=res // 4     # Shapely 2: segmentos por ¼-círculo
    )
    buffers.append(buf)

# --- unión paralela ---
chunks = [buffers[i:i+400] for i in range(0, len(buffers), 400)]
with ThreadPoolExecutor() as pool:
    partes = list(tqdm(pool.map(unary_union, chunks),
                       total=len(chunks), desc="Uniendo"))
calzada = unary_union(partes).buffer(0)   # limpia vacíos

# sellado y simplificación final
calzada = (calzada.buffer(0.2, resolution=4)
                    .buffer(-0.2, resolution=4)
                    .simplify(0.015, preserve_topology=True))

print(f"↳ Buffer+union: {time.perf_counter()-t1:0.1f}s")

# ────────────────── EXPORTACIÓN ──────────────────
gpd.GeoSeries([calzada], crs=crs_utm).to_file(OUT_UTM, driver="GeoJSON")
gpd.GeoSeries([calzada], crs=crs_utm).to_crs(epsg=4326).to_file(OUT_WGS84, driver="GeoJSON")

print("✅ UTM  →", OUT_UTM.resolve())
print("✅ WGS84→", OUT_WGS84.resolve())
print(f"⏱ Total: {time.perf_counter()-t0:0.1f}s")


Descargando:   0%|                                                                              | 0/6 [00:00<?, ?it/s]C:\Users\CA987YS\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.maptiler.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Descargando:  17%|███████████▋                                                          | 1/6 [00:01<00:05,  1.17s/it]C:\Users\CA987YS\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.maptiler.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Descargando:  33%|███████████████████████▎                                              | 2/6 [00:02<0

↳ Descarga: 5.5s


Uniendo: 100%|██████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.18it/s]


↳ Buffer+union: 7.6s
✅ UTM  → C:\Users\CA987YS\01. Proyectos\Cartografia\calzada_buffer.geojson
✅ WGS84→ C:\Users\CA987YS\01. Proyectos\Cartografia\calzada_buffer_wgs84.geojson
⏱ Total: 14.4s


# Mejora 2

In [16]:
#!/usr/bin/env python3
# pip install geopandas shapely vt2geojson mercantile tqdm pyproj pandas numpy requests

import time, math, json, requests, mercantile
import numpy as np, pandas as pd, geopandas as gpd
from pathlib import Path
from tqdm import tqdm
from shapely.geometry import LineString, MultiLineString, GeometryCollection
from shapely.ops import unary_union
from concurrent.futures import ThreadPoolExecutor
from vt2geojson.tools import vt_bytes_to_geojson
from pyproj import CRS, Geod

# ───────── CONFIG ─────────
API_KEY  = "BDxJkUVBgwiy5yhJyLyf"
CENTER   = (14.6398861, -90.5496683)     # lat, lon
RADIUS_M = 1000
ZOOM     = 15
LAYER    = "transportation"
TMP       = Path("ejes.geojson")
OUT_UTM   = Path("calzada_buffer.geojson")
OUT_WGS84 = Path("calzada_buffer_wgs84.geojson")
crs_utm   = CRS.from_epsg(32615)         # Guatemala 15 N

DEFAULT_WIDTH = {
    "motorway":32,"trunk":26,"primary":22,"secondary":18,
    "tertiary":14,"residential":10,"service":8,"footway":4
}
LANE_WIDTH = 2.8          # carril muy delgado
NARROW     = 0.72         # recorte global 40 %

# ───────── DENSIFICAR ─────────
def densify_linestring(ls, max_seg=3.0):
    pts=list(ls.coords); out=[pts[0]]
    for a,b in zip(pts, pts[1:]):
        seg=LineString([a,b])
        n=max(1, math.ceil(seg.length/max_seg))
        for k in range(1,n):
            out.append(seg.interpolate(k/n, normalized=True).coords[0])
        out.append(b)
    return LineString(out)

def densify_geom(geom, max_seg=3.0):
    if isinstance(geom, LineString):
        return densify_linestring(geom, max_seg)
    elif isinstance(geom, MultiLineString):
        return MultiLineString([densify_linestring(ls, max_seg) for ls in geom.geoms])
    return geom

# ───────── CURVATURA ─────────
def _angles(pts):
    angs=[]
    for i in range(1,len(pts)-1):
        p0,p1,p2=pts[i-1],pts[i],pts[i+1]
        v1=(p1[0]-p0[0], p1[1]-p0[1]); v2=(p2[0]-p1[0], p2[1]-p1[1])
        mag=math.hypot(*v1)*math.hypot(*v2)
        if mag==0: continue
        ang=math.degrees(math.acos(max(min((v1[0]*v2[0]+v1[1]*v2[1])/mag,1),-1)))
        angs.append(ang)
    return angs

def curve_score(g):
    if g.is_empty: return 0.0
    angs=[]
    if isinstance(g, LineString):
        angs+=_angles(list(g.coords))
    elif isinstance(g,(MultiLineString,GeometryCollection)):
        for p in g.geoms:
            if isinstance(p,LineString): angs+=_angles(list(p.coords))
    return np.percentile(angs,90) if angs else 0.0

def densify_and_score(g):
    d=densify_geom(g)
    return d, curve_score(d)

# ───────── DESCARGA TESSELAS ─────────
t0=time.perf_counter()
geod=Geod(ellps="WGS84")
lons,lats=[],[]
for az in (0,90,180,270):
    lon,lat,_=geod.fwd(CENTER[1],CENTER[0],az,RADIUS_M)
    lons.append(lon); lats.append(lat)
tiles=list(mercantile.tiles(min(lons),min(lats),max(lons),max(lats),[ZOOM]))
features=[]
for t in tqdm(tiles,desc="Descargando"):
    url=f"https://api.maptiler.com/tiles/v3/{t.z}/{t.x}/{t.y}.pbf?key={API_KEY}"
    features+=vt_bytes_to_geojson(requests.get(url,timeout=15,verify=False).content,
                                  t.x,t.y,t.z,layer=LAYER)["features"]
TMP.write_text(json.dumps({"type":"FeatureCollection","features":features},ensure_ascii=False))
print(f"↳ Descarga: {time.perf_counter()-t0:0.1f}s")

# ───────── GENERAR BUFFER ─────────
t1=time.perf_counter()
gdf=gpd.read_file(TMP).to_crs(crs_utm); idx=gdf.index
width_s=pd.to_numeric(gdf.get("width"),errors="coerce")
lanes_s=pd.to_numeric(gdf.get("lanes"),errors="coerce")
if not isinstance(width_s,pd.Series): width_s=pd.Series(width_s,index=idx)
if not isinstance(lanes_s,pd.Series): lanes_s=pd.Series(lanes_s,index=idx)
class_s=gdf.get("class").reindex(idx)

gdf["computed_width"]=(width_s.fillna(lanes_s*LANE_WIDTH)
                             .fillna(class_s.map(DEFAULT_WIDTH))
                             .fillna(6))  # 6 m mínimo

densified=[]; scores=[]
for g in tqdm(gdf.geometry,desc="Curvature"):
    d,s=densify_and_score(g); densified.append(d); scores.append(s)
gdf["geometry"]=densified; gdf["curve_score"]=scores

# shrink base + curvatura + global
p10,p50,p90=gdf["computed_width"].quantile([.10,.50,.90])
gdf["shrink_factor"]=np.interp(gdf["computed_width"],[p10,p50,p90],[0.90,0.75,0.70])
gdf["curve_factor"]=np.interp(gdf["curve_score"],[0,60],[1.0,0.6]).clip(0.6,1.0)
gdf["eff_shrink"]=gdf["shrink_factor"]*gdf["curve_factor"]*NARROW

buffers=[]
for _,r in tqdm(gdf.iterrows(),total=len(gdf),desc="Buffering"):
    radio=r.computed_width*r.eff_shrink/2
    res=int(np.clip(np.interp(radio*2,[2,40],[8,32]),8,32))
    buffers.append(r.geometry.buffer(radio,cap_style="flat",
                                     join_style="mitre",mitre_limit=3.0,
                                     quad_segs=res//4))

# unión
parts=[unary_union(buffers[i:i+400]) for i in range(0,len(buffers),400)]
calzada=unary_union(parts).buffer(0)
calzada=(calzada.buffer(0.2,resolution=4)
                 .buffer(-0.2,resolution=4)
                 .simplify(0.015,preserve_topology=True))
print(f"↳ Buffer+union: {time.perf_counter()-t1:0.1f}s")

gpd.GeoSeries([calzada],crs=crs_utm).to_file(OUT_UTM,driver="GeoJSON")
gpd.GeoSeries([calzada],crs=crs_utm).to_crs(epsg=4326).to_file(OUT_WGS84,driver="GeoJSON")
print("✅ UTM  →",OUT_UTM.resolve())
print("✅ WGS84→",OUT_WGS84.resolve())
print(f"⏱ Total: {time.perf_counter()-t0:0.1f}s")


Descargando:   0%|                                                                              | 0/6 [00:00<?, ?it/s]C:\Users\CA987YS\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.maptiler.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Descargando:  17%|███████████▋                                                          | 1/6 [00:00<00:02,  1.93it/s]C:\Users\CA987YS\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.maptiler.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Descargando:  33%|███████████████████████▎                                              | 2/6 [00:01<0

↳ Descarga: 7.4s


Buffering: 100%|██████████████████████████████████████████████████████████████████| 511/511 [00:00<00:00, 2178.28it/s]


↳ Buffer+union: 12.0s
✅ UTM  → C:\Users\CA987YS\01. Proyectos\Cartografia\calzada_buffer.geojson
✅ WGS84→ C:\Users\CA987YS\01. Proyectos\Cartografia\calzada_buffer_wgs84.geojson
⏱ Total: 20.7s


# Mejora 3

In [19]:
#!/usr/bin/env python3
# pip install geopandas shapely vt2geojson mercantile tqdm pyproj pandas numpy requests

import time, math, json, requests, mercantile
import numpy as np, pandas as pd, geopandas as gpd
from pathlib import Path
from tqdm import tqdm
from shapely.geometry import LineString, MultiLineString, GeometryCollection
from shapely.ops import unary_union
from concurrent.futures import ThreadPoolExecutor
from vt2geojson.tools import vt_bytes_to_geojson
from pyproj import CRS, Geod

# ───────── CONFIG ─────────
API_KEY  = "BDxJkUVBgwiy5yhJyLyf"
CENTER   = (14.6398861, -90.5496683)     # lat, lon
RADIUS_M = 1000
ZOOM     = 15
LAYER    = "transportation"
TMP       = Path("ejes.geojson")
OUT_UTM   = Path("calzada_buffer.geojson")
OUT_WGS84 = Path("calzada_buffer_wgs84.geojson")
crs_utm   = CRS.from_epsg(32615)         # Guatemala 15 N

DEFAULT_WIDTH = {
    "motorway":32,"trunk":26,"primary":22,"secondary":18,
    "tertiary":14,"residential":10,"service":8,"footway":4
}
LANE_WIDTH = 2.8
NARROW     = 0.72

# ───────── DENSIFICAR ─────────
def densify_linestring(ls, max_seg=3.0):
    pts=list(ls.coords); out=[pts[0]]
    for a,b in zip(pts, pts[1:]):
        seg=LineString([a,b])
        n=max(1, math.ceil(seg.length/max_seg))
        for k in range(1,n):
            out.append(seg.interpolate(k/n, normalized=True).coords[0])
        out.append(b)
    return LineString(out)

def densify_geom(geom, max_seg=3.0):
    if isinstance(geom, LineString):
        return densify_linestring(geom, max_seg)
    elif isinstance(geom, MultiLineString):
        return MultiLineString([densify_linestring(ls, max_seg) for ls in geom.geoms])
    return geom

# ───────── CURVATURA ─────────
def _angles(pts):
    angs=[]
    for i in range(1,len(pts)-1):
        p0,p1,p2=pts[i-1],pts[i],pts[i+1]
        v1=(p1[0]-p0[0], p1[1]-p0[1]); v2=(p2[0]-p1[0], p2[1]-p1[1])
        mag=math.hypot(*v1)*math.hypot(*v2)
        if mag==0: continue
        ang=math.degrees(math.acos(max(min((v1[0]*v2[0]+v1[1]*v2[1])/mag,1),-1)))
        angs.append(ang)
    return angs

def curve_score(g):
    if g.is_empty: return 0.0
    angs=[]
    if isinstance(g, LineString):
        angs+=_angles(list(g.coords))
    elif isinstance(g,(MultiLineString,GeometryCollection)):
        for p in g.geoms:
            if isinstance(p,LineString): angs+=_angles(list(p.coords))
    return np.percentile(angs,90) if angs else 0.0

def densify_and_score(g):
    d=densify_geom(g)
    return d, curve_score(d)

# ───────── DESCARGA TESSELAS ─────────
t0=time.perf_counter()
geod=Geod(ellps="WGS84")
lons,lats=[],[]
for az in (0,90,180,270):
    lon,lat,_=geod.fwd(CENTER[1],CENTER[0],az,RADIUS_M)
    lons.append(lon); lats.append(lat)
tiles=list(mercantile.tiles(min(lons),min(lats),max(lons),max(lats),[ZOOM]))
features=[]
for t in tqdm(tiles,desc="Descargando"):
    url=f"https://api.maptiler.com/tiles/v3/{t.z}/{t.x}/{t.y}.pbf?key={API_KEY}"
    features+=vt_bytes_to_geojson(requests.get(url,timeout=15,verify=False).content,
                                  t.x,t.y,t.z,layer=LAYER)["features"]
TMP.write_text(json.dumps({"type":"FeatureCollection","features":features},ensure_ascii=False))
print(f"↳ Descarga: {time.perf_counter()-t0:0.1f}s")

# ───────── GENERAR BUFFER ─────────
t1=time.perf_counter()
gdf=gpd.read_file(TMP).to_crs(crs_utm); idx=gdf.index
width_s=pd.to_numeric(gdf.get("width"),errors="coerce")
lanes_s=pd.to_numeric(gdf.get("lanes"),errors="coerce")
if not isinstance(width_s,pd.Series): width_s=pd.Series(width_s,index=idx)
if not isinstance(lanes_s,pd.Series): lanes_s=pd.Series(lanes_s,index=idx)
class_s=gdf.get("class").reindex(idx)

gdf["computed_width"]=(width_s.fillna(lanes_s*LANE_WIDTH)
                             .fillna(class_s.map(DEFAULT_WIDTH))
                             .fillna(6))

densified=[]; scores=[]
for g in tqdm(gdf.geometry,desc="Curvature"):
    d,s=densify_and_score(g); densified.append(d); scores.append(s)
gdf["geometry"]=densified; gdf["curve_score"]=scores

# ───────── AJUSTE DE ANCHOS ─────────
p10,p50,p90=gdf["computed_width"].quantile([.10,.50,.90])
gdf["shrink_factor"]=np.interp(gdf["computed_width"],[p10,p50,p90],[0.90,0.75,0.70])
gdf["curve_factor"]=np.interp(gdf["curve_score"],[0,60],[1.0,0.6]).clip(0.6,1.0)

# ### NUEVO — ajuste extra para avenidas ultra-anchas ###
p95 = gdf["computed_width"].quantile(.95)
p99 = gdf["computed_width"].quantile(.99)
WIDE_LIMIT = max(p95, 22)            # umbral de avenida muy ancha
MAX_W      = max(p99, WIDE_LIMIT+8)  # techo de la rampa
WIDE_MIN   = 0.58                    # contracción mínima

extra_narrow = np.interp(
    gdf["computed_width"],
    [WIDE_LIMIT, MAX_W],
    [1.0, WIDE_MIN]
)
# -------------------------------------------------------

gdf["eff_shrink"] = (
    gdf["shrink_factor"]
    * gdf["curve_factor"]
    * NARROW
    * extra_narrow            # ← aplica solo si la vía es muy ancha
)

buffers=[]
for _,r in tqdm(gdf.iterrows(),total=len(gdf),desc="Buffering"):
    radio=r.computed_width*r.eff_shrink/2
    res=int(np.clip(np.interp(radio*2,[2,40],[8,32]),8,32))
    buffers.append(r.geometry.buffer(radio,cap_style="flat",
                                     join_style="mitre",mitre_limit=3.0,
                                     quad_segs=res//4))

# unión
parts=[unary_union(buffers[i:i+400]) for i in range(0,len(buffers),400)]
calzada=unary_union(parts).buffer(0)
calzada=(calzada.buffer(0.2,resolution=4)
                 .buffer(-0.2,resolution=4)
                 .simplify(0.015,preserve_topology=True))
print(f"↳ Buffer+union: {time.perf_counter()-t1:0.1f}s")

gpd.GeoSeries([calzada],crs=crs_utm).to_file(OUT_UTM,driver="GeoJSON")
gpd.GeoSeries([calzada],crs=crs_utm).to_crs(epsg=4326).to_file(OUT_WGS84,driver="GeoJSON")
print("✅ UTM  →",OUT_UTM.resolve())
print("✅ WGS84→",OUT_WGS84.resolve())
print(f"⏱ Total: {time.perf_counter()-t0:0.1f}s")


Descargando:   0%|                                                                              | 0/6 [00:00<?, ?it/s]C:\Users\CA987YS\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.maptiler.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Descargando:  17%|███████████▋                                                          | 1/6 [00:01<00:05,  1.13s/it]C:\Users\CA987YS\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.maptiler.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Descargando:  33%|███████████████████████▎                                              | 2/6 [00:01<0

↳ Descarga: 6.1s


Buffering: 100%|██████████████████████████████████████████████████████████████████| 511/511 [00:00<00:00, 1510.52it/s]


↳ Buffer+union: 13.9s
✅ UTM  → C:\Users\CA987YS\01. Proyectos\Cartografia\calzada_buffer.geojson
✅ WGS84→ C:\Users\CA987YS\01. Proyectos\Cartografia\calzada_buffer_wgs84.geojson
⏱ Total: 21.5s
